# Test ingested data for the monthly run

In [69]:
# VIEWS 3
from viewser import Queryset, Column
from views_mapper2.label_writer import *

## Prepare query for given time period and country

In [70]:
# Call the function

def vid2date(i):
    year=str(ViewsMonth(i).year)
    month=str(ViewsMonth(i).month)
    return year+'/'+month

In [3]:
# Choose year and month below to print the correlated `month_id`

print(ViewsMonth.from_year_month(year=2024, month=10)) 

ViewsMonth(id=538) #=> year:2024, month:10


In [4]:
# Set period to query for

StartOfHistory = 520
EndOfHistory = 550

StartYear = 2010
EndYear = 2026

In [5]:
# Set country to query for
name_date2cid('Nigeria','2017-07-01') # The date here is irrelevant

79

In [67]:

## Country name to country ID
country_id = name_date2cid('Afghanistan','2017-07-01') # The date here is irrelevant

## Create df with identifiers

In [7]:
# Identifiers

identifiers_cm = (Queryset("identifiers", "country_month")

.with_column(Column('year', from_loa = "country_year", from_column = 'year_id'))
.with_column(Column('month', from_loa = "month", from_column = 'month'))
.with_column(Column('isoab', from_loa = "country", from_column = 'isoab'))
.with_column(Column('country', from_loa = "country", from_column = 'name'))

)
      
identifiers_cm = identifiers_cm.publish().fetch()

print(f"A dataset with {len(identifiers_cm.columns)} columns, with "
      f"data between t {min(identifiers_cm.index.get_level_values(0))} "
      f"and {max(identifiers_cm.index.get_level_values(0))}. "
      f"({len(np.unique(identifiers_cm.index.get_level_values(1)))} units)"
     )

Queryset identifiers read successfully 
A dataset with 4 columns, with data between t 1 and 852. (213 units)


In [8]:
identifiers_cm

year  month isoab              country
month_id country_id                                        
1        1           1980      1   GUY               Guyana
         2           1980      1   SUR             Suriname
         3           1980      1   TTO  Trinidad and Tobago
         4           1980      1   VEN            Venezuela
         5           1980      1   WSM                Samoa
...                   ...    ...   ...                  ...
852      242         2050     12   TZA             Tanzania
         243         2050     12   MAR              Morocco
         244         2050     12   MRT           Mauritania
         245         2050     12   SDN                Sudan
         246         2050     12   SSD          South Sudan

[158230 rows x 4 columns]

## Create df with UCDP data

In [62]:
ucdp_data_cm = (Queryset("fatalities002_ucdp_cm", "country_month")

# GED

.with_column(Column("ucdp_ged_sb_best_sum", from_loa = "country_month", from_column = "ged_sb_best_sum_nokgi"))                   
##.with_column(Column("ucdp_ged_os_best_sum", from_loa = "country_month", from_column = "ged_os_best_sum_nokgi"))   
#.with_column(Column("ucdp_ged_ns_best_sum", from_loa = "country_month", from_column = "ged_ns_best_sum_nokgi"))   

)
      
ucdp_data_cm = ucdp_data_cm.publish().fetch()

print(f"A dataset with {len(ucdp_data_cm.columns)} columns, with "
      f"data between t {min(ucdp_data_cm.index.get_level_values(0))} "
      f"and {max(ucdp_data_cm.index.get_level_values(0))}. "
      f"({len(np.unique(ucdp_data_cm.index.get_level_values(1)))} units)"
     )


Queryset fatalities002_ucdp_cm read successfully 
A dataset with 1 columns, with data between t 1 and 852. (213 units)


In [63]:
ucdp_data_cm.describe()


,ucdp_ged_sb_best_sum
count,82432.000000
mean,29.879768
std,701.036143
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,124427.000000


In [64]:
ucdp_data_cm

ucdp_ged_sb_best_sum
month_id country_id                      
1        1                            NaN
         2                            NaN
         3                            NaN
         4                            NaN
         5                            NaN
...                                   ...
852      242                          NaN
         243                          NaN
         244                          NaN
         245                          NaN
         246                          NaN

[158230 rows x 1 columns]

In [65]:
# Merge identifier df with data df

cols = ["country_id", "month_id"]

ucdp_data_cm_w_identifiers = pd.merge(identifiers_cm, ucdp_data_cm, on=cols, how="left")

ucdp_data_cm_w_identifiers

,,year,month,isoab,country,ucdp_ged_sb_best_sum
country_id,month_id,,,,,
1,1,1980,1,GUY,Guyana,NaN
2,1,1980,1,SUR,Suriname,NaN
3,1,1980,1,TTO,Trinidad and Tobago,NaN
4,1,1980,1,VEN,Venezuela,NaN
5,1,1980,1,WSM,Samoa,NaN
...,...,...,...,...,...,...
242,852,2050,12,TZA,Tanzania,NaN
243,852,2050,12,MAR,Morocco,NaN
244,852,2050,12,MRT,Mauritania,NaN


In [68]:
# UPDATED SUBSET + GED

ged_cm_subset= ucdp_data_cm_w_identifiers.query(f'month_id >= {StartOfHistory} & month_id <= {EndOfHistory} & country_id == {country_id}')

ged_cm_subset # displays the subset

year  month isoab      country  ucdp_ged_sb_best_sum
country_id month_id                                                      
133        520       2023      4   AFG  Afghanistan                  47.0
           521       2023      5   AFG  Afghanistan                  14.0
           522       2023      6   AFG  Afghanistan                  30.0
           523       2023      7   AFG  Afghanistan                   6.0
           524       2023      8   AFG  Afghanistan                  24.0
           525       2023      9   AFG  Afghanistan                   9.0
           526       2023     10   AFG  Afghanistan                   4.0
           527       2023     11   AFG  Afghanistan                   8.0
           528       2023     12   AFG  Afghanistan                   5.0
           529       2024      1   AFG  Afghanistan                   8.0
           530       2024      2   AFG  Afghanistan                   1.0
           531       2024      3   AFG  Afghanistan                  53.0
           532       2024      4   AFG  Afghanistan                   4.0
           533       2024      5   AFG  Afghanistan                  32.0
           534       2024      6   AFG  Afghanistan                  13.0
           535       2024      7   AFG  Afghanistan                   9.0
           536       2024      8   AFG  Afghanistan                  13.0
           537       2024      9   AFG  Afghanistan                  24.0
           538       2024     10   AFG  Afghanistan                   9.0
           539       2024     11   AFG  Afghanistan                   7.0
           540       2024     12   AFG  Afghanistan                  90.0
           541       2025      1   AFG  Afghanistan                  11.0
           542       2025      2   AFG  Afghanistan                  25.0
           543       2025      3   AFG  Afghanistan                   8.0
           544       2025      4   AFG  Afghanistan                   1.0
           545       2025      5   AFG  Afghanistan                  15.0
           546       2025      6   AFG  Afghanistan                  12.0
           547       2025      7   AFG  Afghanistan                   4.0
           548       2025      8   AFG  Afghanistan                  16.0
           549       2025      9   AFG  Afghanistan                   NaN
           550       2025     10   AFG  Afghanistan                   NaN

## Create df with ACLED data

In [14]:
acled_data_cm = (Queryset("fatalities002_acled_cm", "country_month")

# ACLED

.with_column(Column("acled_sb_count", from_loa = "country_month", from_column = "acled_sb_count"))                   
.with_column(Column("acled_sb_fat", from_loa = "country_month", from_column = "acled_sb_fat"))                   
.with_column(Column("acled_os_fat", from_loa = "country_month", from_column = "acled_os_fat"))   
.with_column(Column("acled_ns_fat", from_loa = "country_month", from_column = "acled_ns_fat"))   
.with_column(Column("acled_bat_gov_fat", from_loa = "country_month", from_column = "acled_bat_gov_fat"))  
.with_column(Column("acled_bat_reb_fat", from_loa = "country_month", from_column = "acled_bat_reb_fat"))                   
.with_column(Column("acled_pr_count", from_loa = "country_month", from_column = "acled_pr_count")) 
.with_column(Column("acled_prx_count", from_loa = "country_month", from_column = "acled_prx_count")) 
.with_column(Column("acled_prx_fat", from_loa = "country_month", from_column = "acled_prx_fat"))                   

)
      
acled_data_cm = acled_data_cm.publish().fetch()

print(f"A dataset with {len(acled_data_cm.columns)} columns, with "
      f"data between t {min(acled_data_cm.index.get_level_values(0))} "
      f"and {max(acled_data_cm.index.get_level_values(0))}. "
      f"({len(np.unique(acled_data_cm.index.get_level_values(1)))} units)"
     )

Queryset fatalities002_acled_cm read successfully 
A dataset with 9 columns, with data between t 1 and 852. (213 units)


In [15]:
acled_data_cm

acled_sb_count  acled_sb_fat  acled_os_fat  acled_ns_fat  \
month_id country_id                                                             
1        1                      NaN           NaN           NaN           NaN   
         2                      NaN           NaN           NaN           NaN   
         3                      NaN           NaN           NaN           NaN   
         4                      NaN           NaN           NaN           NaN   
         5                      NaN           NaN           NaN           NaN   
...                             ...           ...           ...           ...   
852      242                    NaN           NaN           NaN           NaN   
         243                    NaN           NaN           NaN           NaN   
         244                    NaN           NaN           NaN           NaN   
         245                    NaN           NaN           NaN           NaN   
         246                    NaN           NaN           NaN           NaN   

                     acled_bat_gov_fat  acled_bat_reb_fat  acled_pr_count  \
month_id country_id                                                         
1        1                         NaN                NaN             NaN   
         2                         NaN                NaN             NaN   
         3                         NaN                NaN             NaN   
         4                         NaN                NaN             NaN   
         5                         NaN                NaN             NaN   
...                                ...                ...             ...   
852      242                       NaN                NaN             NaN   
         243                       NaN                NaN             NaN   
         244                       NaN                NaN             NaN   
         245                       NaN                NaN             NaN   
         246                       NaN                NaN             NaN   

                     acled_prx_count  acled_prx_fat  
month_id country_id                                  
1        1                       NaN            NaN  
         2                       NaN            NaN  
         3                       NaN            NaN  
         4                       NaN            NaN  
         5                       NaN            NaN  
...                              ...            ...  
852      242                     NaN            NaN  
         243                     NaN            NaN  
         244                     NaN            NaN  
         245                     NaN            NaN  
         246                     NaN            NaN  

[158230 rows x 9 columns]

In [16]:
acled_data_cm.describe()

,acled_sb_count,acled_sb_fat,acled_os_fat,acled_ns_fat,acled_bat_gov_fat,acled_bat_reb_fat,acled_pr_count,acled_prx_count,acled_prx_fat
count,65415.000000,65415.000000,65415.000000,65415.000000,65415.000000,65415.000000,65415.000000,65415.000000,65415.000000
mean,9.301674,19.622441,8.917282,4.632424,0.936192,0.646885,16.155836,1.435848,0.343163
std,121.037663,296.827877,67.936535,39.141131,19.578186,13.899710,94.057744,8.891148,8.762629
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,5982.000000,39588.000000,5237.000000,2051.000000,2054.000000,2000.000000,6572.000000,455.000000,1311.000000


In [17]:
# Merge identifier df with data df

cols = ["country_id", "month_id"]

acled_data_cm_w_identifiers = pd.merge(identifiers_cm, acled_data_cm, on=cols, how="left")

acled_data_cm_w_identifiers

,,year,month,isoab,country,acled_sb_count,acled_sb_fat,acled_os_fat,acled_ns_fat,acled_bat_gov_fat,acled_bat_reb_fat,acled_pr_count,acled_prx_count,acled_prx_fat
country_id,month_id,,,,,,,,,,,,,
1,1,1980,1,GUY,Guyana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1980,1,SUR,Suriname,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1980,1,TTO,Trinidad and Tobago,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1980,1,VEN,Venezuela,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,1980,1,WSM,Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,852,2050,12,TZA,Tanzania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243,852,2050,12,MAR,Morocco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
244,852,2050,12,MRT,Mauritania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
# Query the data – OLD version

acled_cm_subset= acled_data_cm_w_identifiers.query(f'month_id >= {StartOfHistory} & month_id <= {EndOfHistory} & country_id == {country_id}')

acled_cm_subset # displays the subset


year  month isoab  country  acled_sb_count  acled_sb_fat  \
country_id month_id                                                             
79         520       2023      4   NGA  Nigeria            62.0         269.0   
           521       2023      5   NGA  Nigeria            47.0         121.0   
           522       2023      6   NGA  Nigeria            44.0         309.0   
           523       2023      7   NGA  Nigeria            65.0         267.0   
           524       2023      8   NGA  Nigeria            61.0         181.0   
           525       2023      9   NGA  Nigeria            49.0         222.0   
           526       2023     10   NGA  Nigeria            76.0         300.0   
           527       2023     11   NGA  Nigeria            88.0         373.0   
           528       2023     12   NGA  Nigeria            81.0         253.0   
           529       2024      1   NGA  Nigeria            96.0         293.0   
           530       2024      2   NGA  Nigeria            23.0          60.0   
           531       2024      3   NGA  Nigeria            99.0         320.0   
           532       2024      4   NGA  Nigeria            84.0         445.0   
           533       2024      5   NGA  Nigeria            72.0         202.0   
           534       2024      6   NGA  Nigeria            59.0         290.0   
           535       2024      7   NGA  Nigeria            80.0         293.0   
           536       2024      8   NGA  Nigeria            52.0         259.0   
           537       2024      9   NGA  Nigeria            58.0         222.0   
           538       2024     10   NGA  Nigeria            52.0         149.0   
           539       2024     11   NGA  Nigeria            67.0         351.0   
           540       2024     12   NGA  Nigeria            55.0         228.0   
           541       2025      1   NGA  Nigeria            66.0         847.0   
           542       2025      2   NGA  Nigeria            53.0         194.0   
           543       2025      3   NGA  Nigeria            74.0         455.0   
           544       2025      4   NGA  Nigeria            57.0         279.0   
           545       2025      5   NGA  Nigeria            69.0         448.0   
           546       2025      6   NGA  Nigeria            46.0         336.0   
           547       2025      7   NGA  Nigeria            76.0         302.0   
           548       2025      8   NGA  Nigeria           103.0         436.0   
           549       2025      9   NGA  Nigeria           143.0         530.0   
           550       2025     10   NGA  Nigeria             NaN           NaN   

                     acled_os_fat  acled_ns_fat  acled_bat_gov_fat  \
country_id month_id                                                  
79         520              258.0         305.0                0.0   
           521              263.0         227.0                0.0   
           522              401.0         134.0                0.0   
           523              239.0         112.0                0.0   
           524              207.0         293.0                0.0   
           525              169.0          97.0                0.0   
           526              221.0          96.0                0.0   
           527              226.0         244.0                0.0   
           528              380.0         194.0                0.0   
           529              302.0         144.0                0.0   
           530              102.0         124.0                0.0   
           531              362.0         180.0                0.0   
           532              276.0         380.0                0.0   
           533              482.0         255.0                0.0   
           534              265.0         289.0                0.0   
           535              205.0         168.0                0.0   
           536              269.0         307.0                0.0   
           537   

## Create df with Topics data

In [19]:
topics_data_cm = (Queryset("fatalities002_topics_cm", "country_month")

# GED

.with_column(Column("randomly_selected__column", from_loa = "country_month", from_column = "topic_ste_theta13"))                   
)
      
topics_data_cm = topics_data_cm.publish().fetch()

print(f"A dataset with {len(topics_data_cm.columns)} columns, with "
      f"data between t {min(topics_data_cm.index.get_level_values(0))} "
      f"and {max(topics_data_cm.index.get_level_values(0))}. "
      f"({len(np.unique(topics_data_cm.index.get_level_values(1)))} units)"
     )

Queryset fatalities002_topics_cm read successfully 
A dataset with 1 columns, with data between t 1 and 852. (213 units)


In [20]:
topics_data_cm.describe()

,randomly_selected__column
count,70912.000000
mean,0.099292
std,0.081505
min,0.000817
25%,0.039290
50%,0.078221
75%,0.136689
max,0.693780


In [21]:
# Merge identifier df with data df

cols = ["country_id", "month_id"]

topics_data_cm_w_identifiers = pd.merge(identifiers_cm, topics_data_cm, on=cols, how="left")

topics_data_cm_w_identifiers

,,year,month,isoab,country,randomly_selected__column
country_id,month_id,,,,,
1,1,1980,1,GUY,Guyana,NaN
2,1,1980,1,SUR,Suriname,NaN
3,1,1980,1,TTO,Trinidad and Tobago,NaN
4,1,1980,1,VEN,Venezuela,NaN
5,1,1980,1,WSM,Samoa,NaN
...,...,...,...,...,...,...
242,852,2050,12,TZA,Tanzania,NaN
243,852,2050,12,MAR,Morocco,NaN
244,852,2050,12,MRT,Mauritania,NaN


In [22]:
# Query the data

topics_cm_subset= topics_data_cm_w_identifiers.query(f'month_id >= {StartOfHistory} & month_id <= {EndOfHistory} & country_id == {country_id}')

topics_cm_subset # displays the subset

year  month isoab  country  randomly_selected__column
country_id month_id                                                       
79         520       2023      4   NGA  Nigeria                   0.006906
           521       2023      5   NGA  Nigeria                   0.005308
           522       2023      6   NGA  Nigeria                   0.009012
           523       2023      7   NGA  Nigeria                   0.023991
           524       2023      8   NGA  Nigeria                   0.026111
           525       2023      9   NGA  Nigeria                   0.053595
           526       2023     10   NGA  Nigeria                   0.029629
           527       2023     11   NGA  Nigeria                   0.012815
           528       2023     12   NGA  Nigeria                   0.021678
           529       2024      1   NGA  Nigeria                   0.022688
           530       2024      2   NGA  Nigeria                   0.014848
           531       2024      3   NGA  Nigeria                   0.008337
           532       2024      4   NGA  Nigeria                   0.025571
           533       2024      5   NGA  Nigeria                   0.030608
           534       2024      6   NGA  Nigeria                   0.007114
           535       2024      7   NGA  Nigeria                   0.012400
           536       2024      8   NGA  Nigeria                   0.026699
           537       2024      9   NGA  Nigeria                   0.011847
           538       2024     10   NGA  Nigeria                   0.025146
           539       2024     11   NGA  Nigeria                   0.038014
           540       2024     12   NGA  Nigeria                   0.025269
           541       2025      1   NGA  Nigeria                   0.049897
           542       2025      2   NGA  Nigeria                   0.011995
           543       2025      3   NGA  Nigeria                   0.031263
           544       2025      4   NGA  Nigeria                   0.035174
           545       2025      5   NGA  Nigeria                   0.019883
           546       2025      6   NGA  Nigeria                   0.026677
           547       2025      7   NGA  Nigeria                   0.020889
           548       2025      8   NGA  Nigeria                   0.035797
           549       2025      9   NGA  Nigeria                   0.026527
           550       2025     10   NGA  Nigeria                        NaN

## Create df with V-Dem data

In [23]:
vdem_data_cm = (Queryset("fatalities002_vdem_cm", "country_year")

# V-Dem

.with_column(Column("vdem_v2x_libdem", from_loa = "country_year", from_column = "vdem_v2x_gender"))                   
)
      
vdem_data_cm = vdem_data_cm.publish().fetch()

print(f"A dataset with {len(vdem_data_cm.columns)} columns, with "
      f"data between t {min(vdem_data_cm.index.get_level_values(0))} "
      f"and {max(vdem_data_cm.index.get_level_values(0))}. "
      f"({len(np.unique(vdem_data_cm.index.get_level_values(1)))} units)"
     )

Queryset fatalities002_vdem_cm read successfully                                            
A dataset with 1 columns, with data between t 1900 and 2050. (220 units)


In [24]:
vdem_data_cm.describe()

,vdem_v2x_libdem
count,7372.000000
mean,0.653571
std,0.214394
min,0.022000
25%,0.496000
50%,0.691000
75%,0.836000
max,0.962000


In [25]:
vdem_data_cm

vdem_v2x_libdem
year_id country_id                 
1900    4                       NaN
        7                       NaN
        8                       NaN
        10                      NaN
        11                      NaN
...                             ...
2050    242                     NaN
        243                     NaN
        244                     NaN
        245                     NaN
        246                     NaN

[20392 rows x 1 columns]

In [26]:
# Fix discrepancies in column names

vdem_data_cm = (
    vdem_data_cm
    .reset_index()
    .rename(columns={'year_id': 'year'})
    .set_index(['year', 'country_id'])
)

In [27]:
vdem_data_cm


vdem_v2x_libdem
year country_id                 
1900 4                       NaN
     7                       NaN
     8                       NaN
     10                      NaN
     11                      NaN
...                          ...
2050 242                     NaN
     243                     NaN
     244                     NaN
     245                     NaN
     246                     NaN

[20392 rows x 1 columns]

In [28]:
# Merge identifier df with data df

cols = ["country_id", "year"]

vdem_data_cm_w_identifiers = pd.merge(identifiers_cm, vdem_data_cm, on=cols, how="left")

vdem_data_cm_w_identifiers

,year,month,isoab,country,vdem_v2x_libdem
country_id,,,,,
1,1980,1,GUY,Guyana,0.736
2,1980,1,SUR,Suriname,0.603
3,1980,1,TTO,Trinidad and Tobago,0.639
4,1980,1,VEN,Venezuela,0.722
5,1980,1,WSM,Samoa,NaN
...,...,...,...,...,...
242,2050,12,TZA,Tanzania,NaN
243,2050,12,MAR,Morocco,NaN
244,2050,12,MRT,Mauritania,NaN


In [29]:
# Query the data

vdem_data_cm= vdem_data_cm.query(f'year >= 2023 & year <= 2025 & country_id == {country_id}')

vdem_data_cm # displays the subset

,,vdem_v2x_libdem
year,country_id,
2023,79,0.712
2024,79,0.687
2025,79,NaN


# WDI

In [30]:
wdi_data_cm = (Queryset("fatalities002_wdi_cm", "country_year")

# V-Dem

.with_column(Column("wb_wdi_ny_gdp_pcap_kd", from_loa = "country_year", from_column = "wdi_ny_gdp_pcap_kd"))
.with_column(Column("wb_wdi_sp_dyn_le00_in", from_loa = "country_year", from_column = "wdi_sp_dyn_le00_in"))                   
.with_column(Column("wb_wdi_se_prm_nenr", from_loa = "country_year", from_column = "wdi_se_prm_nenr"))                   
.with_column(Column("wb_wdi_sp_pop_totl", from_loa = "country_year", from_column = "wdi_sp_pop_totl"))                   
.with_column(Column("wb_wdi_sp_dyn_imrt_in", from_loa = "country_year", from_column = "wdi_sp_dyn_imrt_in"))                   
)
      
wdi_data_cm = wdi_data_cm.publish().fetch()

print(f"A dataset with {len(wdi_data_cm.columns)} columns, with "
      f"data between t {min(wdi_data_cm.index.get_level_values(0))} "
      f"and {max(wdi_data_cm.index.get_level_values(0))}. "
      f"({len(np.unique(wdi_data_cm.index.get_level_values(1)))} units)"
     )

Queryset fatalities002_wdi_cm read successfully                                             
A dataset with 5 columns, with data between t 1900 and 2050. (220 units)


In [31]:
wdi_data_cm.describe()

,wb_wdi_ny_gdp_pcap_kd,wb_wdi_sp_dyn_le00_in,wb_wdi_se_prm_nenr,wb_wdi_sp_pop_totl,wb_wdi_sp_dyn_imrt_in
count,7990.000000,8091.000000,3714.000000,8.284000e+03,8014.000000
mean,11972.708354,67.531654,86.111799,3.365178e+07,37.160507
std,18908.800507,9.912602,16.141461,1.273240e+08,35.103148
min,164.700417,12.158000,14.308450,7.366000e+03,1.400000
25%,1398.250286,61.167000,84.198770,1.395300e+06,9.800000
50%,4023.453783,69.517000,92.259495,6.803706e+06,23.900000
75%,13943.841232,74.890500,96.426705,2.182829e+07,55.700000
max,224582.449752,86.372000,100.000000,1.450936e+09,274.000000


In [32]:
wdi_data_cm

wb_wdi_ny_gdp_pcap_kd  wb_wdi_sp_dyn_le00_in  \
year_id country_id                                                 
1900    4                             NaN                    NaN   
        7                             NaN                    NaN   
        8                             NaN                    NaN   
        10                            NaN                    NaN   
        11                            NaN                    NaN   
...                                   ...                    ...   
2050    242                           NaN                    NaN   
        243                           NaN                    NaN   
        244                           NaN                    NaN   
        245                           NaN                    NaN   
        246                           NaN                    NaN   

                    wb_wdi_se_prm_nenr  wb_wdi_sp_pop_totl  \
year_id country_id                                           
1900    4                          NaN                 NaN   
        7                          NaN                 NaN   
        8                          NaN                 NaN   
        10                         NaN                 NaN   
        11                         NaN                 NaN   
...                                ...                 ...   
2050    242                        NaN                 NaN   
        243                        NaN                 NaN   
        244                        NaN                 NaN   
        245                        NaN                 NaN   
        246                        NaN                 NaN   

                    wb_wdi_sp_dyn_imrt_in  
year_id country_id                         
1900    4                             NaN  
        7                             NaN  
        8                             NaN  
        10                            NaN  
        11                            NaN  
...                                   ...  
2050    242                           NaN  
        243                           NaN  
        244                           NaN  
        245                           NaN  
        246                           NaN  

[20392 rows x 5 columns]

In [33]:
# Fix discrepancies in column names

wdi_data_cm = (
    wdi_data_cm
    .reset_index()
    .rename(columns={'year_id': 'year'})
    .set_index(['year', 'country_id'])
)

In [34]:
# Merge identifier df with data df

cols = ["country_id", "year"]

wdi_data_cm_w_identifiers = pd.merge(identifiers_cm, wdi_data_cm, on=cols, how="left")

wdi_data_cm_w_identifiers

,year,month,isoab,country,wb_wdi_ny_gdp_pcap_kd,wb_wdi_sp_dyn_le00_in,wb_wdi_se_prm_nenr,wb_wdi_sp_pop_totl,wb_wdi_sp_dyn_imrt_in
country_id,,,,,,,,,
1,1980,1,GUY,Guyana,3247.672768,60.808,NaN,774367.0,59.0
2,1980,1,SUR,Suriname,7110.251415,60.654,NaN,375377.0,NaN
3,1980,1,TTO,Trinidad and Tobago,8622.350611,66.507,86.2554,1121744.0,36.8
4,1980,1,VEN,Venezuela,NaN,68.179,NaN,15258218.0,36.0
5,1980,1,WSM,Samoa,2642.834121,64.961,NaN,164322.0,33.2
...,...,...,...,...,...,...,...,...,...
242,2050,12,TZA,Tanzania,NaN,NaN,NaN,NaN,NaN
243,2050,12,MAR,Morocco,NaN,NaN,NaN,NaN,NaN
244,2050,12,MRT,Mauritania,NaN,NaN,NaN,NaN,NaN


In [35]:
# Query the data

wdi_data_cm= wdi_data_cm.query(f'year >= {StartYear} & year <= {EndYear} & country_id == {country_id}')

wdi_data_cm # displays the subset

,,wb_wdi_ny_gdp_pcap_kd,wb_wdi_sp_dyn_le00_in,wb_wdi_se_prm_nenr,wb_wdi_sp_pop_totl,wb_wdi_sp_dyn_imrt_in
year,country_id,,,,,
2010,79,2315.466765,51.315,64.13783,166642886.0,74.0
2011,79,2370.976571,51.490,NaN,171379598.0,73.0
2012,79,2403.653879,51.688,NaN,176200625.0,72.0
2013,79,2495.341066,51.827,NaN,181049443.0,71.4
2014,79,2583.615570,51.940,NaN,185896915.0,70.8
2015,79,2585.733607,51.939,NaN,190671878.0,70.2
2016,79,2481.814904,52.189,NaN,195443700.0,69.5
2017,79,2441.712364,52.397,NaN,200254579.0,68.6
2018,79,2431.778578,52.669,NaN,204938755.0,67.4


# FAOSTAT CPI

In [36]:
faostat_cpi_cm = (Queryset("fatalities002_faostat_cpi_cm", "country_month")

# FAOSTAT CPI

.with_column(Column("fao_cpi_consumer_prices_food_indices", from_loa = "country_month", from_column = "consumer_prices_food_indices"))
.with_column(Column("fao_cpi_consumer_prices_general_indices", from_loa = "country_month", from_column = "consumer_prices_general_indices"))                   
.with_column(Column("fao_cpi_food_price_inflation", from_loa = "country_month", from_column = "food_price_inflation"))  

)

faostat_cpi_cm = faostat_cpi_cm.publish().fetch()
print(f"A dataset with {len(faostat_cpi_cm.columns)} columns, with "
      f"data between t {min(faostat_cpi_cm.index.get_level_values(0))} "
      f"and {max(faostat_cpi_cm.index.get_level_values(0))}. "
      f"({len(np.unique(faostat_cpi_cm.index.get_level_values(1)))} units)"
     )



100%|██████████| 1.54M/1.54M [00:00<00:00, 17.3MB/s]


Queryset fatalities002_faostat_cpi_cm read successfully 
A dataset with 3 columns, with data between t 1 and 852. (213 units)


In [37]:
faostat_cpi_cm.describe()

,fao_cpi_consumer_prices_food_indices,fao_cpi_consumer_prices_general_indices,fao_cpi_food_price_inflation
count,5.435000e+04,5.435000e+04,54350.000000
mean,1.054854e+09,7.562135e+08,49.518698
std,4.729305e+10,3.401095e+10,3005.711673
min,0.000000e+00,0.000000e+00,-24.982988
25%,6.731791e+01,7.262530e+01,1.170049
50%,9.363922e+01,9.490000e+01,4.186098
75%,1.079294e+02,1.069909e+02,8.902237
max,2.890921e+12,2.097637e+12,371537.503265


In [38]:
faostat_cpi_cm

fao_cpi_consumer_prices_food_indices  \
month_id country_id                                         
1        1                                            NaN   
         2                                            NaN   
         3                                            NaN   
         4                                            NaN   
         5                                            NaN   
...                                                   ...   
852      242                                          NaN   
         243                                          NaN   
         244                                          NaN   
         245                                          NaN   
         246                                          NaN   

                     fao_cpi_consumer_prices_general_indices  \
month_id country_id                                            
1        1                                               NaN   
         2                                               NaN   
         3                                               NaN   
         4                                               NaN   
         5                                               NaN   
...                                                      ...   
852      242                                             NaN   
         243                                             NaN   
         244                                             NaN   
         245                                             NaN   
         246                                             NaN   

                     fao_cpi_food_price_inflation  
month_id country_id                                
1        1                                    NaN  
         2                                    NaN  
         3                                    NaN  
         4                                    NaN  
         5                                    NaN  
...                                           ...  
852      242                                  NaN  
         243                                  NaN  
         244                                  NaN  
         245                                  NaN  
         246                                  NaN  

[158230 rows x 3 columns]

In [39]:
# Merge identifier df with data df

cols = ["country_id", "month_id"]

faostat_cpi_cm_cm_w_identifiers = pd.merge(identifiers_cm, faostat_cpi_cm, on=cols, how="left")

faostat_cpi_cm_cm_w_identifiers

,,year,month,isoab,country,fao_cpi_consumer_prices_food_indices,fao_cpi_consumer_prices_general_indices,fao_cpi_food_price_inflation
country_id,month_id,,,,,,,
1,1,1980,1,GUY,Guyana,NaN,NaN,NaN
2,1,1980,1,SUR,Suriname,NaN,NaN,NaN
3,1,1980,1,TTO,Trinidad and Tobago,NaN,NaN,NaN
4,1,1980,1,VEN,Venezuela,NaN,NaN,NaN
5,1,1980,1,WSM,Samoa,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
242,852,2050,12,TZA,Tanzania,NaN,NaN,NaN
243,852,2050,12,MAR,Morocco,NaN,NaN,NaN
244,852,2050,12,MRT,Mauritania,NaN,NaN,NaN


In [40]:
# Query the data

faostat_cpi = faostat_cpi_cm_cm_w_identifiers.query(f'month_id >= {StartOfHistory} & month_id <= {EndOfHistory} & country_id == {country_id}')

faostat_cpi # displays the subset

year  month isoab  country  \
country_id month_id                               
79         520       2023      4   NGA  Nigeria   
           521       2023      5   NGA  Nigeria   
           522       2023      6   NGA  Nigeria   
           523       2023      7   NGA  Nigeria   
           524       2023      8   NGA  Nigeria   
           525       2023      9   NGA  Nigeria   
           526       2023     10   NGA  Nigeria   
           527       2023     11   NGA  Nigeria   
           528       2023     12   NGA  Nigeria   
           529       2024      1   NGA  Nigeria   
           530       2024      2   NGA  Nigeria   
           531       2024      3   NGA  Nigeria   
           532       2024      4   NGA  Nigeria   
           533       2024      5   NGA  Nigeria   
           534       2024      6   NGA  Nigeria   
           535       2024      7   NGA  Nigeria   
           536       2024      8   NGA  Nigeria   
           537       2024      9   NGA  Nigeria   
           538       2024     10   NGA  Nigeria   
           539       2024     11   NGA  Nigeria   
           540       2024     12   NGA  Nigeria   
           541       2025      1   NGA  Nigeria   
           542       2025      2   NGA  Nigeria   
           543       2025      3   NGA  Nigeria   
           544       2025      4   NGA  Nigeria   
           545       2025      5   NGA  Nigeria   
           546       2025      6   NGA  Nigeria   
           547       2025      7   NGA  Nigeria   
           548       2025      8   NGA  Nigeria   
           549       2025      9   NGA  Nigeria   
           550       2025     10   NGA  Nigeria   

                     fao_cpi_consumer_prices_food_indices  \
country_id month_id                                         
79         520                                 357.433216   
           521                                 365.208124   
           522                                 373.898578   
           523                                 386.754872   
           524                                 401.641449   
           525                                 411.429007   
           526                                 419.287600   
           527                                 429.395536   
           528                                 441.019307   
           529                                 455.115158   
           530                                 472.327048   
           531                                 489.345524   
           532                                 501.629111   
           533                                 513.116163   
           534                                 526.183446   
           535                                 539.194405   
           536                                 552.016473   
           537                                 566.585974   
           538                                 583.221698   
           539                                 600.588106   
           540                                 616.527938   
           541                                 588.855323   
           542                                 601.100958   
           543                                 614.411431   
           544                                 613.879012   
           545                                 624.527391   
           546                                 643.162053   
           547                                        NaN   
           548                                        NaN   
           549                                        NaN   
           550                                        NaN   

                     fao_cpi_consumer_prices_general_indices  \
country_id month_id                                            
79         520                                    310.313185   
           521                                    316.339181   
           522                                    323.072274   
   

In [41]:
# Query the data

faostat_cpi = faostat_cpi_cm_cm_w_identifiers.query(f'month_id >= {StartOfHistory} & month_id <= {EndOfHistory} & country_id == {country_id}')

faostat_cpi # displays the subset

year  month isoab  country  \
country_id month_id                               
79         520       2023      4   NGA  Nigeria   
           521       2023      5   NGA  Nigeria   
           522       2023      6   NGA  Nigeria   
           523       2023      7   NGA  Nigeria   
           524       2023      8   NGA  Nigeria   
           525       2023      9   NGA  Nigeria   
           526       2023     10   NGA  Nigeria   
           527       2023     11   NGA  Nigeria   
           528       2023     12   NGA  Nigeria   
           529       2024      1   NGA  Nigeria   
           530       2024      2   NGA  Nigeria   
           531       2024      3   NGA  Nigeria   
           532       2024      4   NGA  Nigeria   
           533       2024      5   NGA  Nigeria   
           534       2024      6   NGA  Nigeria   
           535       2024      7   NGA  Nigeria   
           536       2024      8   NGA  Nigeria   
           537       2024      9   NGA  Nigeria   
           538       2024     10   NGA  Nigeria   
           539       2024     11   NGA  Nigeria   
           540       2024     12   NGA  Nigeria   
           541       2025      1   NGA  Nigeria   
           542       2025      2   NGA  Nigeria   
           543       2025      3   NGA  Nigeria   
           544       2025      4   NGA  Nigeria   
           545       2025      5   NGA  Nigeria   
           546       2025      6   NGA  Nigeria   
           547       2025      7   NGA  Nigeria   
           548       2025      8   NGA  Nigeria   
           549       2025      9   NGA  Nigeria   
           550       2025     10   NGA  Nigeria   

                     fao_cpi_consumer_prices_food_indices  \
country_id month_id                                         
79         520                                 357.433216   
           521                                 365.208124   
           522                                 373.898578   
           523                                 386.754872   
           524                                 401.641449   
           525                                 411.429007   
           526                                 419.287600   
           527                                 429.395536   
           528                                 441.019307   
           529                                 455.115158   
           530                                 472.327048   
           531                                 489.345524   
           532                                 501.629111   
           533                                 513.116163   
           534                                 526.183446   
           535                                 539.194405   
           536                                 552.016473   
           537                                 566.585974   
           538                                 583.221698   
           539                                 600.588106   
           540                                 616.527938   
           541                                 588.855323   
           542                                 601.100958   
           543                                 614.411431   
           544                                 613.879012   
           545                                 624.527391   
           546                                 643.162053   
           547                                        NaN   
           548                                        NaN   
           549                                        NaN   
           550                                        NaN   

                     fao_cpi_consumer_prices_general_indices  \
country_id month_id                                            
79         520                                    310.313185   
           521                                    316.339181   
           522                                    323.072274   
   

# FAOSTAT FSEC

In [42]:
faostat_fsec_cm = (Queryset("fatalities002_faostat_fsec_cm", "country_year")

# FAOSTAT FSEC

.with_column(Column("fao_fsec_gdp_pc_ppp", from_loa = "country_year", from_column = "gdp_pc_ppp")) # not updated
.with_column(Column("fao_fsec_pol_stability", from_loa = "country_year", from_column = "pol_stability"))                   
.with_column(Column("fao_fsec_pcap_fprod_var", from_loa = "country_year", from_column = "pcap_fprod_var")) # not updated                  
.with_column(Column("fao_fsec_pcap_fsupply_var", from_loa = "country_year", from_column = "pcap_fsupply_var"))                   
.with_column(Column("fao_fsec_pct_pop_basicdrink", from_loa = "country_year", from_column = "pct_pop_basicdrink"))                   
.with_column(Column("fao_fsec_pct_pop_basicsani", from_loa = "country_year", from_column = "pct_pop_basicsani"))                   
.with_column(Column("fao_fsec_pct_und5_wasting", from_loa = "country_year", from_column = "pct_und5_wasting"))                   
.with_column(Column("fao_fsec_pct_und5_stunted", from_loa = "country_year", from_column = "pct_und5_stunted"))                   
.with_column(Column("fao_fsec_pct_und5_overweight", from_loa = "country_year", from_column = "pct_und5_overweight"))                   
.with_column(Column("fao_fsec_prev_adult_obesity", from_loa = "country_year", from_column = "prev_adult_obesity"))                   
.with_column(Column("fao_fsec_prev_repr_anemia", from_loa = "country_year", from_column = "prev_repr_anemia"))                   
.with_column(Column("fao_fsec_prev_infant_bfeed", from_loa = "country_year", from_column = "prev_infant_bfeed"))                   
.with_column(Column("fao_fsec_rail_density", from_loa = "country_year", from_column = "rail_density"))                   
.with_column(Column("fao_fsec_pct_pop_safedrink", from_loa = "country_year", from_column = "pct_pop_safedrink"))                   
.with_column(Column("fao_fsec_pct_pop_safesani", from_loa = "country_year", from_column = "pct_pop_safesani"))                   
.with_column(Column("fao_fsec_prev_lowbweight", from_loa = "country_year", from_column = "prev_lowbweight"))                   
.with_column(Column("fao_fsec_avg_adequate_diet", from_loa = "country_year", from_column = "avg_adequate_diet"))                   
.with_column(Column("fao_fsec_avg_fprod_value", from_loa = "country_year", from_column = "avg_fprod_value"))  # not updated                 
.with_column(Column("fao_fsec_kcal_pcap_day", from_loa = "country_year", from_column = "kcal_pcap_day"))    # not updated               
.with_column(Column("fao_fsec_kcal_pcap_day_cerotu", from_loa = "country_year", from_column = "kcal_pcap_day_cerotu"))           # not updated        
.with_column(Column("fao_fsec_avg_protein_pcap_day", from_loa = "country_year", from_column = "avg_protein_pcap_day"))                   
.with_column(Column("fao_fsec_avg_animalprotein_pcap_day", from_loa = "country_year", from_column = "avg_animalprotein_pcap_day"))                   
.with_column(Column("fao_fsec_pop_undernourished", from_loa = "country_year", from_column = "pop_undernourished"))                   
.with_column(Column("fao_fsec_pct_cereal_import", from_loa = "country_year", from_column = "pct_cereal_import"))                   
.with_column(Column("fao_fsec_pct_arable_land", from_loa = "country_year", from_column = "pct_arable_land"))                    
.with_column(Column("fao_fsec_pct_fimport_merch", from_loa = "country_year", from_column = "pct_fimport_merch"))                   
.with_column(Column("fao_fsec_pct_severe_finsecurity", from_loa = "country_year", from_column = "pct_severe_finsecurity"))                   
.with_column(Column("fao_fsec_pct_modsevere_finsecurity", from_loa = "country_year", from_column = "pct_modsevere_finsecurity"))                   
.with_column(Column("fao_fsec_pop_severe_finsecurity", from_loa = "country_year", from_column = "pop_severe_finsecurity"))                   
.with_column(Column("fao_fsec_pop_modsevere_finsecurity", from_loa = "country_year", from_column = "pop_modsevere_finsecurity"))                   
.with_column(Column("fao_fsec_avg_energy_req", from_loa = "country_year", from_column = "avg_energy_req"))                   
.with_column(Column("fao_fsec_avg_fatsupply", from_loa = "country_year", from_column = "avg_fatsupply"))                   
.with_column(Column("fao_fsec_hcc_coeff_dist", from_loa = "country_year", from_column = "hcc_coeff_dist"))                   
.with_column(Column("fao_fsec_caloric_loss_retail", from_loa = "country_year", from_column = "caloric_loss_retail"))      
.with_column(Column("fao_fsec_min_energy_req", from_loa = "country_year", from_column = "min_energy_req"))                  
)
      
faostat_fsec_cm = faostat_fsec_cm.publish().fetch()

print(f"A dataset with {len(faostat_fsec_cm.columns)} columns, with "
      f"data between t {min(faostat_fsec_cm.index.get_level_values(0))} "
      f"and {max(faostat_fsec_cm.index.get_level_values(0))}. "
      f"({len(np.unique(faostat_fsec_cm.index.get_level_values(1)))} units)"
     )

Queryset fatalities002_faostat_fsec_cm read successfully                                              
A dataset with 35 columns, with data between t 1900 and 2050. (220 units)


In [43]:
faostat_fsec_cm.describe()

,fao_fsec_gdp_pc_ppp,fao_fsec_pol_stability,fao_fsec_pcap_fprod_var,fao_fsec_pcap_fsupply_var,fao_fsec_pct_pop_basicdrink,fao_fsec_pct_pop_basicsani,fao_fsec_pct_und5_wasting,fao_fsec_pct_und5_stunted,fao_fsec_pct_und5_overweight,fao_fsec_prev_adult_obesity,...,fao_fsec_pct_fimport_merch,fao_fsec_pct_severe_finsecurity,fao_fsec_pct_modsevere_finsecurity,fao_fsec_pop_severe_finsecurity,fao_fsec_pop_modsevere_finsecurity,fao_fsec_avg_energy_req,fao_fsec_avg_fatsupply,fao_fsec_hcc_coeff_dist,fao_fsec_caloric_loss_retail,fao_fsec_min_energy_req
count,4074.000000,4430.000000,4074.000000,4465.000000,4379.000000,4371.000000,3700.000000,4602.000000,4609.000000,4429.000000,...,4424.000000,4074.000000,4310.000000,4074.000000,4074.000000,4797.000000,4207.000000,4469.000000,4825.000000,4797.000000
mean,16544.019710,-0.071018,9.537776,36.789698,81.797922,69.345665,1.276162,17.557019,5.691972,16.245834,...,50.384494,1.616846,7.903109,0.410898,1.428598,2253.142381,73.906251,0.238367,2.749670,1745.326454
std,19285.848438,0.944569,12.554413,29.721887,23.656166,32.007576,3.181954,15.444509,4.744479,11.456554,...,242.004309,6.190713,17.515641,1.970131,6.604933,488.259418,42.742506,0.082507,1.123914,374.767778
min,0.000000,-3.310000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2875.275000,-0.640000,1.100000,19.000000,71.200000,43.400000,0.000000,3.800000,2.200000,6.300000,...,6.000000,0.000000,0.000000,0.000000,0.000000,2218.000000,47.700000,0.210000,2.170000,1730.000000
50%,9735.800000,0.000000,5.900000,31.000000,93.000000,84.500000,0.000000,13.700000,4.900000,16.600000,...,13.000000,0.000000,0.000000,0.000000,0.000000,2341.000000,71.100000,0.250000,2.870000,1815.000000
75%,23126.400000,0.680000,12.700000,48.000000,99.000000,97.250000,0.000000,29.400000,8.400000,22.700000,...,35.000000,0.000000,5.400000,0.000000,0.000000,2473.000000,100.700000,0.280000,3.410000,1905.000000
max,114889.000000,1.760000,107.700000,286.000000,99.000000,99.000000,21.500000,62.300000,29.300000,71.700000,...,9809.000000,65.400000,89.800000,43.000000,116.000000,2718.000000,183.600000,0.490000,7.010000,2077.000000


In [44]:
# Fix discrepancies in column names

faostat_fsec_cm = (
    faostat_fsec_cm
    .reset_index()
    .rename(columns={'year_id': 'year'})
    .set_index(['year', 'country_id'])
)

In [45]:
faostat_fsec_cm

fao_fsec_gdp_pc_ppp  fao_fsec_pol_stability  \
year country_id                                                
1900 4                           NaN                     NaN   
     7                           NaN                     NaN   
     8                           NaN                     NaN   
     10                          NaN                     NaN   
     11                          NaN                     NaN   
...                              ...                     ...   
2050 242                         NaN                     NaN   
     243                         NaN                     NaN   
     244                         NaN                     NaN   
     245                         NaN                     NaN   
     246                         NaN                     NaN   

                 fao_fsec_pcap_fprod_var  fao_fsec_pcap_fsupply_var  \
year country_id                                                       
1900 4                               NaN                        NaN   
     7                               NaN                        NaN   
     8                               NaN                        NaN   
     10                              NaN                        NaN   
     11                              NaN                        NaN   
...                                  ...                        ...   
2050 242                             NaN                        NaN   
     243                             NaN                        NaN   
     244                             NaN                        NaN   
     245                             NaN                        NaN   
     246                             NaN                        NaN   

                 fao_fsec_pct_pop_basicdrink  fao_fsec_pct_pop_basicsani  \
year country_id                                                            
1900 4                                   NaN                         NaN   
     7                                   NaN                         NaN   
     8                                   NaN                         NaN   
     10                                  NaN                         NaN   
     11                                  NaN                         NaN   
...                                      ...                         ...   
2050 242                                 NaN                         NaN   
     243                                 NaN                         NaN   
     244                                 NaN                         NaN   
     245                                 NaN                         NaN   
     246                                 NaN                         NaN   

                 fao_fsec_pct_und5_wasting  fao_fsec_pct_und5_stunted  \
year country_id                                                         
1900 4                                 NaN                        NaN   
     7                                 NaN                        NaN   
     8                                 NaN                        NaN   
     10                                NaN                        NaN   
     11                                NaN                        NaN   
...                                    ...                        ...   
2050 242                               NaN                        NaN   
     243                               NaN                        NaN   
     244                               NaN                        NaN   
     245                               NaN                        NaN   
     246                               NaN                        NaN   

                 fao_fsec_pct_und5_overweight  fao_fsec_prev_adult_obesity  \
year country_id                                                              
1900 4                                    NaN                          NaN   
     7                                    NaN                          NaN   
     8   

In [46]:
# Merge identifier df with data df

cols = ["country_id", "year"]

faostat_fsec_cm_w_identifiers = pd.merge(identifiers_cm, faostat_fsec_cm, on=cols, how="left")

faostat_fsec_cm_w_identifiers

,year,month,isoab,country,fao_fsec_gdp_pc_ppp,fao_fsec_pol_stability,fao_fsec_pcap_fprod_var,fao_fsec_pcap_fsupply_var,fao_fsec_pct_pop_basicdrink,fao_fsec_pct_pop_basicsani,...,fao_fsec_pct_fimport_merch,fao_fsec_pct_severe_finsecurity,fao_fsec_pct_modsevere_finsecurity,fao_fsec_pop_severe_finsecurity,fao_fsec_pop_modsevere_finsecurity,fao_fsec_avg_energy_req,fao_fsec_avg_fatsupply,fao_fsec_hcc_coeff_dist,fao_fsec_caloric_loss_retail,fao_fsec_min_energy_req
country_id,,,,,,,,,,,,,,,,,,,,,
1,1980,1,GUY,Guyana,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1980,1,SUR,Suriname,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1980,1,TTO,Trinidad and Tobago,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1980,1,VEN,Venezuela,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1980,1,WSM,Samoa,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,2050,12,TZA,Tanzania,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243,2050,12,MAR,Morocco,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
244,2050,12,MRT,Mauritania,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
# Query the data

faostat_fsec_cm = faostat_fsec_cm.query(f'year >= {StartYear} & year <= {EndYear} & country_id == {country_id}')

faostat_fsec_cm # displays the subset

,,fao_fsec_gdp_pc_ppp,fao_fsec_pol_stability,fao_fsec_pcap_fprod_var,fao_fsec_pcap_fsupply_var,fao_fsec_pct_pop_basicdrink,fao_fsec_pct_pop_basicsani,fao_fsec_pct_und5_wasting,fao_fsec_pct_und5_stunted,fao_fsec_pct_und5_overweight,fao_fsec_prev_adult_obesity,...,fao_fsec_pct_fimport_merch,fao_fsec_pct_severe_finsecurity,fao_fsec_pct_modsevere_finsecurity,fao_fsec_pop_severe_finsecurity,fao_fsec_pop_modsevere_finsecurity,fao_fsec_avg_energy_req,fao_fsec_avg_fatsupply,fao_fsec_hcc_coeff_dist,fao_fsec_caloric_loss_retail,fao_fsec_min_energy_req
year,country_id,,,,,,,,,,,,,,,,,,,,,
2010,79,4932.3,-2.21,15.9,14.0,60.0,34.5,0.0,38.8,2.8,6.7,...,6.0,0.0,0.0,0.0,0.0,2154.0,59.9,0.28,3.20,1694.0
2011,79,5056.9,-1.96,15.3,24.0,61.8,35.2,10.2,38.4,2.6,7.0,...,7.0,0.0,0.0,0.0,0.0,2154.0,57.3,0.28,3.26,1694.0
2012,79,5131.4,-2.04,12.6,27.0,63.5,36.0,0.0,38.0,2.5,7.4,...,6.0,0.0,0.0,0.0,0.0,2154.0,57.3,0.28,3.25,1694.0
2013,79,5329.1,-2.09,12.7,24.0,65.3,36.7,0.0,37.6,2.4,7.7,...,7.0,0.0,0.0,0.0,0.0,2154.0,56.7,0.28,3.30,1694.0
2014,79,5516.4,-2.13,10.0,32.0,67.0,37.5,7.9,37.3,2.4,8.1,...,7.0,6.6,36.5,11.9,66.1,2156.0,56.3,0.28,3.30,1695.0
2015,79,5514.8,-1.93,10.3,29.0,68.8,38.3,7.3,37.0,2.4,8.5,...,9.0,6.6,36.5,11.9,66.1,2157.0,55.3,0.28,3.30,1696.0
2016,79,5284.9,-1.88,11.6,30.0,70.5,39.2,10.8,36.7,2.4,8.9,...,10.0,6.6,36.5,11.9,66.1,2158.0,54.6,0.28,3.30,1697.0
2017,79,5190.4,-2.00,6.9,31.0,72.3,40.0,0.0,36.5,2.4,0.0,...,14.0,9.3,40.1,17.3,74.5,2161.0,53.7,0.28,3.30,1698.0
2018,79,5155.1,-2.10,0.0,35.0,73.0,42.0,6.8,34.4,1.5,10.6,...,10.0,12.1,47.1,23.1,83.2,2203.0,58.5,0.28,3.42,1712.0


# FAOSTAT PP 

In [48]:
faostat_pp_cm = (Queryset("fatalities002_faostat_pp_cm", "country_month")

# FAOSTAT FSEC

.with_column(Column("fao_pp_meat_price", from_loa = "country_month", from_column = "meat_price")) 
.with_column(Column("fao_pp_milk_price", from_loa = "country_month", from_column = "milk_price"))                   
.with_column(Column("fao_pp_mp_price", from_loa = "country_month", from_column = "mp_price"))                  
.with_column(Column("fao_pp_sugar_price", from_loa = "country_month", from_column = "sugar_price"))                   
.with_column(Column("fao_pp_wheat_price", from_loa = "country_month", from_column = "wheat_price"))                   
)
       
faostat_pp_cm = faostat_pp_cm.publish().fetch()

print(f"A dataset with {len(faostat_pp_cm.columns)} columns, with "
      f"data between t {min(faostat_pp_cm.index.get_level_values(0))} "
      f"and {max(faostat_pp_cm.index.get_level_values(0))}. "
      f"({len(np.unique(faostat_pp_cm.index.get_level_values(1)))} units)"
     )

Queryset fatalities002_faostat_pp_cm read successfully 
A dataset with 5 columns, with data between t 1 and 852. (213 units)


In [49]:
faostat_pp_cm.describe()

,fao_pp_meat_price,fao_pp_milk_price,fao_pp_mp_price,fao_pp_sugar_price,fao_pp_wheat_price
count,6.129200e+04,5.931300e+04,7.364900e+04,6.003300e+04,6.310700e+04
mean,1.140176e+05,2.028885e+04,5.941063e+04,5.416694e+03,4.708488e+02
std,3.489718e+06,1.546260e+06,2.017824e+06,1.881021e+05,3.083140e+04
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,7.333333e-01,0.000000e+00,0.000000e+00
max,5.623194e+08,2.880138e+08,3.505188e+08,1.920282e+07,4.498558e+06


In [50]:
faostat_pp_cm

fao_pp_meat_price  fao_pp_milk_price  fao_pp_mp_price  \
month_id country_id                                                          
1        1                         NaN                NaN              NaN   
         2                         NaN                NaN              NaN   
         3                         NaN                NaN              NaN   
         4                         NaN                NaN              NaN   
         5                         NaN                NaN              NaN   
...                                ...                ...              ...   
852      242                       NaN                NaN              NaN   
         243                       NaN                NaN              NaN   
         244                       NaN                NaN              NaN   
         245                       NaN                NaN              NaN   
         246                       NaN                NaN              NaN   

                     fao_pp_sugar_price  fao_pp_wheat_price  
month_id country_id                                          
1        1                          NaN                 NaN  
         2                          NaN                 NaN  
         3                          NaN                 NaN  
         4                          NaN                 NaN  
         5                          NaN                 NaN  
...                                 ...                 ...  
852      242                        NaN                 NaN  
         243                        NaN                 NaN  
         244                        NaN                 NaN  
         245                        NaN                 NaN  
         246                        NaN                 NaN  

[158230 rows x 5 columns]

In [51]:
# Merge identifier df with data df

cols = ["country_id", "month_id"]

faostat_pp_cm_cm_w_identifiers = pd.merge(identifiers_cm, faostat_pp_cm, on=cols, how="left")

faostat_pp_cm_cm_w_identifiers

,,year,month,isoab,country,fao_pp_meat_price,fao_pp_milk_price,fao_pp_mp_price,fao_pp_sugar_price,fao_pp_wheat_price
country_id,month_id,,,,,,,,,
1,1,1980,1,GUY,Guyana,NaN,NaN,NaN,NaN,NaN
2,1,1980,1,SUR,Suriname,NaN,NaN,NaN,NaN,NaN
3,1,1980,1,TTO,Trinidad and Tobago,NaN,NaN,NaN,NaN,NaN
4,1,1980,1,VEN,Venezuela,NaN,NaN,NaN,NaN,NaN
5,1,1980,1,WSM,Samoa,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
242,852,2050,12,TZA,Tanzania,NaN,NaN,NaN,NaN,NaN
243,852,2050,12,MAR,Morocco,NaN,NaN,NaN,NaN,NaN
244,852,2050,12,MRT,Mauritania,NaN,NaN,NaN,NaN,NaN


In [52]:
# Query the data

faostat_pp_cm = faostat_pp_cm_cm_w_identifiers.query(f'month_id >= {StartOfHistory} & month_id <= {EndOfHistory} & country_id == {country_id}')

faostat_pp_cm # displays the subset

year  month isoab  country  fao_pp_meat_price  \
country_id month_id                                                  
79         520       2023      4   NGA  Nigeria                NaN   
           521       2023      5   NGA  Nigeria                NaN   
           522       2023      6   NGA  Nigeria                NaN   
           523       2023      7   NGA  Nigeria                NaN   
           524       2023      8   NGA  Nigeria                NaN   
           525       2023      9   NGA  Nigeria                NaN   
           526       2023     10   NGA  Nigeria                NaN   
           527       2023     11   NGA  Nigeria                NaN   
           528       2023     12   NGA  Nigeria      475834.290909   
           529       2024      1   NGA  Nigeria                NaN   
           530       2024      2   NGA  Nigeria                NaN   
           531       2024      3   NGA  Nigeria                NaN   
           532       2024      4   NGA  Nigeria                NaN   
           533       2024      5   NGA  Nigeria                NaN   
           534       2024      6   NGA  Nigeria                NaN   
           535       2024      7   NGA  Nigeria                NaN   
           536       2024      8   NGA  Nigeria                NaN   
           537       2024      9   NGA  Nigeria                NaN   
           538       2024     10   NGA  Nigeria                NaN   
           539       2024     11   NGA  Nigeria                NaN   
           540       2024     12   NGA  Nigeria                NaN   
           541       2025      1   NGA  Nigeria                NaN   
           542       2025      2   NGA  Nigeria                NaN   
           543       2025      3   NGA  Nigeria                NaN   
           544       2025      4   NGA  Nigeria                NaN   
           545       2025      5   NGA  Nigeria                NaN   
           546       2025      6   NGA  Nigeria                NaN   
           547       2025      7   NGA  Nigeria                NaN   
           548       2025      8   NGA  Nigeria                NaN   
           549       2025      9   NGA  Nigeria                NaN   
           550       2025     10   NGA  Nigeria                NaN   

                     fao_pp_milk_price  fao_pp_mp_price  fao_pp_sugar_price  \
country_id month_id                                                           
79         520                     NaN              NaN                 NaN   
           521                     NaN              NaN                 NaN   
           522                     NaN              NaN                 NaN   
           523                     NaN              NaN                 NaN   
           524                     NaN              NaN                 NaN   
           525                     NaN              NaN                 NaN   
           526                     NaN              NaN                 NaN   
           527                     NaN              NaN                 NaN   
           528                 41817.3    183527.685714             20302.8   
           529                     NaN              NaN                 NaN   
           530                     NaN              NaN                 NaN   
           531                     NaN              NaN                 NaN   
           532                     NaN              NaN                 NaN   
           533                     NaN              NaN                 NaN   
           534                     NaN              NaN                 NaN   
           535                     NaN              NaN                 NaN   
           536                     NaN              NaN                 NaN   
           537                     NaN              NaN                 NaN   
           538                     NaN              NaN                 NaN   
           539                     NaN             

In [53]:
print("All done!")

All done!
